# Dask jobqueue example

## What is Dask jobqueue? (<https://jobqueue.dask.org/>)

* deploys Dask workers on typical HPC job queueing systems

## Monte-Carlo estimate with multiple Dask batch job workers

We define a Dask jobqueue cluster with Dask workers that each have 8 CPUs and 48 GB of memory.

In [1]:
import dask, dask.distributed
import dask_jobqueue

In [2]:
cluster = dask_jobqueue.SLURMCluster(

    # Dask worker size
    cores=8, memory='48GB',
    processes=1, # Dask workers per job
    
    # SLURM job script things
    queue='cluster', walltime='00:15:00',
    
    # Dask worker network and temporary storage
    interface='ib0', local_directory='$TMPDIR'
)

client = dask.distributed.Client(cluster)

In [3]:
cluster.scale(jobs=1)

In [4]:
client

Client Scheduler: tcp://172.18.4.100:33939 Dashboard: http://172.18.4.100:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


### What is a jobqueue cluster?
The above is all we need to specify to run the computation on compute node Dask workers. 
Let's have a look at what's happening in the background.

In [5]:
!squeue -u $USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
             55461   cluster dask-wor smomw122 PD       0:00      1 (None) 
             55371   cluster jupyterl smomw122  R      18:01      1 neshcl100 


In [6]:
print(cluster.job_script())

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p cluster
#SBATCH -n 1
#SBATCH --cpus-per-task=8
#SBATCH --mem=45G
#SBATCH -t 00:15:00

/gxfs_home/geomar/smomw122/miniconda3/envs/dask_jobqueue_workshop/bin/python -m distributed.cli.dask_worker tcp://172.18.4.100:33939 --nthreads 8 --memory-limit 48.00GB --name dummy-name --nanny --death-timeout 60 --local-directory $TMPDIR --interface ib0 --protocol tcp://



In [7]:
client

Client Scheduler: tcp://172.18.4.100:33939 Dashboard: http://172.18.4.100:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


### Let's scale up the cluster

In [8]:
cluster.scale(jobs=8)

In [9]:
!squeue -u $USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
             55461   cluster dask-wor smomw122 PD       0:00      1 (None) 
             55462   cluster dask-wor smomw122 PD       0:00      1 (None) 
             55371   cluster jupyterl smomw122  R      18:02      1 neshcl100 


In [10]:
client

Client Scheduler: tcp://172.18.4.100:33939 Dashboard: http://172.18.4.100:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


### From here everything is the same as with LocalCluster

In [11]:
import numpy, dask.array

def calculate_pi(size_in_bytes, number_of_chunks):
    
    """Calculate pi using a Monte Carlo method."""
    
    array_shape = (int(size_in_bytes / 8 / 2), 2)
    chunk_size = (int(array_shape[0] / number_of_chunks), 2)
    
    # 2D random positions array using dask.array
    xy = dask.array.random.uniform(
        low=0.0, high=1.0, size=array_shape,
        # specify chunk size, i.e. task number
        chunks=chunk_size )
  
    xy_inside_circle = (xy ** 2).sum(axis=1) < 1 # boolean

    pi = 4 * xy_inside_circle.sum() / xy_inside_circle.size
    
    # start Dask calculation
    pi = pi.compute()

    print(f"\nfrom {xy.nbytes / 1e9} GB randomly chosen positions")
    print(f"   pi estimate: {pi}")
    print(f"   pi error: {abs(pi - numpy.pi)}\n")
    # display(xy)
    
    return pi

### Let's calculate again...

In [12]:
%time pi = calculate_pi(size_in_bytes=10_000_000_000, number_of_chunks=100) # 10 GB


from 10.0 GB randomly chosen positions
   pi estimate: 3.1416435008
   pi error: 5.084721020676142e-05

CPU times: user 219 ms, sys: 27.4 ms, total: 246 ms
Wall time: 3.21 s


In [13]:
%time pi = calculate_pi(size_in_bytes=100_000_000_000, number_of_chunks=250) # 100 GB


from 100.0 GB randomly chosen positions
   pi estimate: 3.14162127552
   pi error: 2.8621930206806923e-05

CPU times: user 469 ms, sys: 56.2 ms, total: 525 ms
Wall time: 5.05 s


In [14]:
%time pi = calculate_pi(size_in_bytes=1_000_000_000_000, number_of_chunks=2_000) # 1 TB


from 1000.0 GB randomly chosen positions
   pi estimate: 3.14159954144
   pi error: 6.88785020708238e-06

CPU times: user 4.97 s, sys: 431 ms, total: 5.4 s
Wall time: 46.5 s


### We can easily scale down the cluster

In [15]:
cluster.scale(jobs=2)

In [16]:
!squeue -u $USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
             55464   cluster dask-wor smomw122 CG       0:55      1 neshcl101 
             55474   cluster dask-wor smomw122 PD       0:00      1 (Resources) 
             55475   cluster dask-wor smomw122 PD       0:00      1 (Priority) 
             55470   cluster dask-wor smomw122  R       0:18      1 neshcl213 
             55471   cluster dask-wor smomw122  R       0:18      1 neshcl306 
             55472   cluster dask-wor smomw122  R       0:18      1 neshcl322 
             55473   cluster dask-wor smomw122  R       0:18      1 neshcl323 
             55461   cluster dask-wor smomw122  R       0:55      1 neshcl251 
             55462   cluster dask-wor smomw122  R       0:55      1 neshcl266 
             55463   cluster dask-wor smomw122  R       0:55      1 neshcl100 
             55465   cluster dask-wor smomw122  R       0:55      1 neshcl103 
             55466   cluster dask-wor smom

### And we can scale up the cluster whenever needed

In [17]:
cluster.scale(jobs=16)

In [18]:
!squeue -u $USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
             55462   cluster dask-wor smomw122 CG       0:55      1 neshcl266 
             55463   cluster dask-wor smomw122 CG       0:55      1 neshcl100 
             55464   cluster dask-wor smomw122 CG       0:55      1 neshcl101 
             55468   cluster dask-wor smomw122 CG       0:55      1 neshcl253 
             55474   cluster dask-wor smomw122 PD       0:00      1 (Resources) 
             55475   cluster dask-wor smomw122 PD       0:00      1 (Priority) 
             55470   cluster dask-wor smomw122  R       0:18      1 neshcl213 
             55471   cluster dask-wor smomw122  R       0:18      1 neshcl306 
             55472   cluster dask-wor smomw122  R       0:18      1 neshcl322 
             55473   cluster dask-wor smomw122  R       0:18      1 neshcl323 
             55461   cluster dask-wor smomw122  R       0:55      1 neshcl251 
             55465   cluster dask-wor smom

In [19]:
client

Client Scheduler: tcp://172.18.4.100:33939 Dashboard: http://172.18.4.100:8787/status,Cluster Workers: 8 Cores: 64 Memory: 384.00 GB


### Let's calculate again...

In [26]:
# %time pi = calculate_pi(size_in_bytes=1_000_000_000_000, number_of_chunks=2_000) # 1 TB

In [21]:
# %time pi = calculate_pi(size_in_bytes=10_000_000_000_000, number_of_chunks=10_000) # 10 TB

### Note, we could also adaptively scale the jobqueue cluster!

Dask jobqueue is able to scale total worker number based on problem size. You can also specify a target duration.

In [22]:
cluster.adapt(
    minimum_jobs=2, maximum_jobs=16,
)

In [23]:
%time pi = calculate_pi(size_in_bytes=10_000_000_000, number_of_chunks=100) # 10 GB


from 10.0 GB randomly chosen positions
   pi estimate: 3.1415523008
   pi error: 4.0352789793196564e-05

CPU times: user 254 ms, sys: 7.25 ms, total: 261 ms
Wall time: 3.07 s


In [24]:
%time pi = calculate_pi(size_in_bytes=1_000_000_000_000, number_of_chunks=1_000) # 1 TB


from 1000.0 GB randomly chosen positions
   pi estimate: 3.141595556288
   pi error: 2.902698206685983e-06

CPU times: user 3.08 s, sys: 284 ms, total: 3.37 s
Wall time: 48 s


In [25]:
# %time pi = calculate_pi(size_in_bytes=10_000_000_000_000, number_of_chunks=10_000) # 10 TB